In [1]:
import os
from langchain.agents import AgentType, initialize_agent, create_pandas_dataframe_agent
from langchain.agents.agent_toolkits import create_vectorstore_agent, VectorStoreToolkit, VectorStoreInfo, create_python_agent
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DataFrameLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.tools import HumanInputRun
from langchain.tools import Tool
from langchain.tools.python.tool import PythonREPLTool
from langchain.utilities import SerpAPIWrapper
from langchain.vectorstores import Chroma

import pandas as pd

In [2]:
from getpass import getpass
import os

OPENAI_API_KEY = getpass()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY



In [3]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=OPENAI_API_KEY)


In [4]:
# simple test
from langchain.llms import OpenAI
from langchain import PromptTemplate, LLMChain

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [5]:
df = pd.read_csv("samples/data.csv")
loader = DataFrameLoader(df, page_content_column="description")
data = loader.load()

In [6]:
openai_embeddings = OpenAIEmbeddings()

In [7]:
vector_store = Chroma.from_documents(data, embedding=openai_embeddings, persist_directory="../../Downloads/vectorstores")
vector_store.persist()

In [9]:
def get_input() -> str:
    display("Insert your text. Enter 'q' or CMD+D toed.")
    contents = []
    while True:
        try:
            line = input()

        except EOFError:
            break
        if line == "q":
            break
        contents.append(line)
    return "\n".join(contents)


In [10]:
search = SerpAPIWrapper()

python_repl = create_python_agent(llm=llm,
                                  tool=PythonREPLTool,
                                  verbose=True)

metadata_search = RetrievalQA.from_chain_type(llm=llm,
                                              chain_type="stuff",
                                              retriever=vector_store.as_retriever())

human_input = HumanInputRun(input_func=get_input)

panda_agent = create_pandas_dataframe_agent(ChatOpenAI(temperature=0), df, verbose=True)

vectorstore_info = VectorStoreInfo(
    name="oink", #name of the input data?
    description="decribe inout date",
    vectorstore=vector_store
)

toolkit = VectorStoreToolkit(vectorstore_info=vectorstore_info)
agent_executor = create_vectorstore_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)


AttributeError: type object 'PythonREPLTool' has no attribute 'name'